This example finds attention heads that move information from variable types to the corresponding variable name in python code. It uses the `FunctionFinder` token finder to find the relevant tokens in the code, then the  `ActivationAnalyzer` class to find heads that match the criteria.

## Setup

In [1]:
from transformer_lens import HookedTransformer
from llm_inspect import TokenDisplayer


llm = HookedTransformer.from_pretrained("EleutherAI/pythia-2.8b-deduped-v0")

C:\Users\UKGC\PycharmProjects\TokenFinder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model EleutherAI/pythia-2.8b-deduped-v0 into HookedTransformer


In [2]:
code = """# This is an irrelevant comment
def add(a: int, b: int, c: str) -> int:
    return str(a + b) + c

def subtract(a: int, b: int) -> int:
    return a - b
"""

input_tokens = llm.tokenizer.tokenize(code, add_special_tokens=True)
input_token_ids = llm.tokenizer.encode(code, add_special_tokens=True, return_tensors="pt")

print(input_tokens)
print(input_token_ids)

['<|endoftext|>', '#', 'ĠThis', 'Ġis', 'Ġan', 'Ġirrelevant', 'Ġcomment', 'Ċ', 'def', 'Ġadd', '(', 'a', ':', 'Ġint', ',', 'Ġb', ':', 'Ġint', ',', 'Ġc', ':', 'Ġstr', ')', 'Ġ->', 'Ġint', ':', 'Ċ', '    ', 'return', 'Ġstr', '(', 'a', 'Ġ+', 'Ġb', ')', 'Ġ+', 'Ġc', 'Ċ', 'Ċ', 'def', 'Ġsubtract', '(', 'a', ':', 'Ġint', ',', 'Ġb', ':', 'Ġint', ')', 'Ġ->', 'Ġint', ':', 'Ċ', '    ', 'return', 'Ġa', 'Ġ-', 'Ġb', 'Ċ']
tensor([[    0,     4,   831,   310,   271, 19124,  4385,   187,  1545,   823,
             9,    66,    27,   540,    13,   270,    27,   540,    13,   260,
            27,  1213,    10,  5204,   540,    27,   187, 50274,  2309,  1213,
             9,    66,   559,   270,    10,   559,   260,   187,   187,  1545,
         43444,     9,    66,    27,   540,    13,   270,    27,   540,    10,
          5204,   540,    27,   187, 50274,  2309,   247,   428,   270,   187]])


In [3]:
_, activation_cache = llm.run_with_cache(input_token_ids)

In [4]:
token_displayer = TokenDisplayer.create_for_tokenizer(llm.tokenizer)

## Find variable type-variable name pairs using FunctionFinder

In [5]:
from llm_inspect import Token
from dataclasses import dataclass

@dataclass
class VariableNameTypePair:
    name_token: Token
    type_tokens: list[Token]

In [6]:
from llm_inspect import AttentionHeadFinder, FunctionFinder

function_finder = FunctionFinder.create_from_tokenizer(code, llm.tokenizer)
activation_analyzer = AttentionHeadFinder.create_from_tokenizer(llm.tokenizer, input_tokens, activation_cache)

In [7]:
function = function_finder.find_function_scope("add")

print(f"Function name token: {function.function_name_token}")
print(f"Parameters: {function.parameters}")
print(f"Return type token: {function.return_type_token}")

Function name token: Token(index=9, value='Ġadd')
Parameters: [FunctionParameter(name=Token(index=11, value='a'), type=Token(index=13, value='Ġint')), FunctionParameter(name=Token(index=15, value='Ġb'), type=Token(index=17, value='Ġint')), FunctionParameter(name=Token(index=19, value='Ġc'), type=Token(index=21, value='Ġstr'))]
Return type token: Token(index=24, value='Ġint')


In [8]:
print("Function scope:")
token_displayer.html_for_scope_with_context(function.function_scope)

Function scope:


In [9]:
print("Function body scope:")
token_displayer.html_for_scope_with_context(function.body_scope)

Function body scope:


In [10]:
a_variable_token = function.body_scope.find_first("a", allow_space_prefix=True)
b_variable_token = function.body_scope.find_first("b", allow_space_prefix=True)
c_variable_token = function.body_scope.find_first("c", allow_space_prefix=True)

token_displayer.html_for_token_with_context(a_variable_token)

### Find heads that move information from variable types to variable names

In [12]:
from llm_inspect import AttentionHead


a_matching_heads = activation_analyzer.find_heads_where_query_looks_at_value(a_variable_token, function.parameters[0].type)
b_matching_heads = activation_analyzer.find_heads_where_query_looks_at_value(b_variable_token, function.parameters[1].type)
c_matching_heads = activation_analyzer.find_heads_where_query_looks_at_value(c_variable_token, function.parameters[2].type)

type_moving_heads = AttentionHead.intersection([
    a_matching_heads,
    b_matching_heads,
    c_matching_heads
])

print(f"Found {len(type_moving_heads)} head(s) that move information from the type token to the variable token.")

Found 2 head(s) that move information from the type token to the variable token.


## Visualise

In [13]:
from llm_inspect import TokenDisplayer


token_displayer = TokenDisplayer.create_for_tokenizer(llm.tokenizer)

In [14]:
print(f"Head {type_moving_heads[0]}:")

token_displayer.html_for_token_attention(
    input_tokens,
    activation_cache,
    type_moving_heads[0],
)

Head 12.21:


What's interesting is that the same head we found that moves variable type information in the python code also moves information from the type to the variable name in Java code, even though python and java have different syntax for declaring variables.

In [15]:
java_code = """// This is an irrelevant comment
public class Main {
    public static void main(String[] args) {
        int a = 1;
        int b = 2;
        String c = "3";
        System.out.println(a + b + c);
    }
}
"""

java_input_tokens = llm.tokenizer.tokenize(java_code, add_special_tokens=True)
java_input_ids = llm.tokenizer.encode(java_code, add_special_tokens=True, return_tensors="pt")
_, java_activation_cache = llm.run_with_cache(java_input_ids)

token_displayer.html_for_token_attention(
    java_input_tokens,
    java_activation_cache.remove_batch_dim(),
    type_moving_heads[0],
)